In [1]:
import requests
import brownie
import os

In [2]:
brownie.network.connect('mainnet')

In [3]:
delegator_contract = "0x469788fE6E9E9681C6ebF3bF78e7Fd26Fc015446"
delegator_contract_obj = brownie.network.contract.Contract(delegator_contract)

In [4]:
vlCVX_contract = "0xD18140b4B819b895A3dba5442F959fA44994AF50"
vlCVX_contract_obj = brownie.network.contract.Contract(vlCVX_contract)

In [5]:
block_info_response = requests.get("https://api.blockcypher.com/v1/eth/main")
current_block = block_info_response.json()['height']

In [6]:
query = f"https://api.etherscan.io/api?module=account&action=txlist&address={delegator_contract}&startblock=0&endblock=9999999999&sort=asc&page=1&offset=10000&apikey={os.environ['ETHERSCAN_API_KEY']}"
etherscan_api_response = requests.post(query)
etherscan_api_response

<Response [200]>

In [7]:
vote_delegation_txes = etherscan_api_response.json()['result']
len(vote_delegation_txes)

513

In [8]:
delegator_contract_obj.setDelegate.decode_input(vote_delegation_txes[0]['input'])

[0x676e6f7369730000000000000000000000000000000000000000000000000000,
 '0xeF8305E140ac520225DAf050e2f71d5fBcC543e7']

In [16]:
delegate_cvx_stake = {}
for tx in vote_delegation_txes:

    tx_receipt = brownie.network.transaction.TransactionReceipt(tx['hash'])
    delegator_addr = tx['from']
    delegator_locked_supply = vlCVX_contract_obj.lockedBalanceOf(delegator_addr) * 1e-18

    if delegator_locked_supply == 0.0:
        continue

    if tx_receipt.fn_name == "setDelegate":

        delegate_addr = delegator_contract_obj.setDelegate.decode_input(tx['input'])[1]

        if delegate_addr not in delegate_cvx_stake.keys():
            delegate_cvx_stake[delegate_addr] = delegator_locked_supply
        else:
            delegate_cvx_stake[delegate_addr] += delegator_locked_supply

    elif tx_receipt.fn_name == "clearDelegate":

        delegated_addr = tx_receipt.events['ClearDelegate']['delegate']

        if delegated_addr not in delegate_cvx_stake.keys():
            delegate_cvx_stake[delegated_addr] = -delegator_locked_supply
        else:
            delegate_cvx_stake[delegated_addr] -= delegator_locked_supply

delegate_cvx_stake

{'0xD549A42b5dd6E06d6145ba65F99df4a0f282AdEf': 3013.288,
 '0xde1E6A7ED0ad3F61D531a8a78E83CcDdbd6E0c49': 598507.6229116425,
 '0x947B7742C403f20e5FaCcDAc5E092C943E7D0277': 79390.15317149487}

'0x9381741a8ada2b21ccacc5a920a65e8c35d0fa94'